In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [2]:
df = pd.read_csv("GYM.csv")

X = df[["Gender", "Goal", "BMI Category"]]
y = df[["Exercise Schedule", "Meal Plan"]]
df

,Gender,Goal,BMI Category,Exercise Schedule,Meal Plan
0,Female,muscle_gain,Normal weight,"Moderate cardio, Strength training, and 5000 s...",Balanced diet with moderate protein and carboh...
1,Male,fat_burn,Underweight,"Light weightlifting, Yoga, and 2000 steps walking","High-calorie, protein-rich diet: Whole milk, p..."
2,Male,muscle_gain,Normal weight,"Moderate cardio, Strength training, and 5000 s...",Balanced diet with moderate protein and carboh...
3,Male,muscle_gain,Overweight,"High-intensity interval training (HIIT), Cardi...","Low-carb, high-fiber diet: Avocado, grilled fi..."
4,Female,muscle_gain,Normal weight,"Moderate cardio, Strength training, and 5000 s...",Balanced diet with moderate protein and carboh...
...,...,...,...,...,...
79995,Male,fat_burn,Normal weight,"Moderate cardio, Strength training, and 5000 s...",Balanced diet with moderate protein and carboh...
79996,Female,fat_burn,Underweight,"Light weightlifting, Yoga, and 2000 steps walking","High-calorie, protein-rich diet: Whole milk, p..."
79997,Female,muscle_gain,Obesity,"Low-impact cardio, Swimming, and 10000 steps w...","Low-calorie, nutrient-dense diet with portion ..."
79998,Male,fat_burn,Normal weight,"Moderate cardio, Strength training, and 5000 s...",Balanced diet with moderate protein and carboh...


In [3]:
X

,Gender,Goal,BMI Category
0,Female,muscle_gain,Normal weight
1,Male,fat_burn,Underweight
2,Male,muscle_gain,Normal weight
3,Male,muscle_gain,Overweight
4,Female,muscle_gain,Normal weight
...,...,...,...
79995,Male,fat_burn,Normal weight
79996,Female,fat_burn,Underweight
79997,Female,muscle_gain,Obesity
79998,Male,fat_burn,Normal weight


In [4]:
y

,Exercise Schedule,Meal Plan
0,"Moderate cardio, Strength training, and 5000 s...",Balanced diet with moderate protein and carboh...
1,"Light weightlifting, Yoga, and 2000 steps walking","High-calorie, protein-rich diet: Whole milk, p..."
2,"Moderate cardio, Strength training, and 5000 s...",Balanced diet with moderate protein and carboh...
3,"High-intensity interval training (HIIT), Cardi...","Low-carb, high-fiber diet: Avocado, grilled fi..."
4,"Moderate cardio, Strength training, and 5000 s...",Balanced diet with moderate protein and carboh...
...,...,...
79995,"Moderate cardio, Strength training, and 5000 s...",Balanced diet with moderate protein and carboh...
79996,"Light weightlifting, Yoga, and 2000 steps walking","High-calorie, protein-rich diet: Whole milk, p..."
79997,"Low-impact cardio, Swimming, and 10000 steps w...","Low-calorie, nutrient-dense diet with portion ..."
79998,"Moderate cardio, Strength training, and 5000 s...",Balanced diet with moderate protein and carboh...


In [5]:
exercise_encoder = LabelEncoder()
meal_encoder = LabelEncoder()

y_exercise = exercise_encoder.fit_transform(y["Exercise Schedule"])
y_meal = meal_encoder.fit_transform(y["Meal Plan"])

ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
X_encoded = ohe.fit_transform(X)

In [6]:
X_train, X_test, y_ex_train, y_ex_test, y_me_train, y_me_test = train_test_split(
    X_encoded,
    y_exercise,
    y_meal,
    test_size=0.2,
    random_state=42
)

In [7]:
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)

y_ex_train = torch.tensor(y_ex_train, dtype=torch.long)
y_ex_test = torch.tensor(y_ex_test, dtype=torch.long)

y_me_train = torch.tensor(y_me_train, dtype=torch.long)
y_me_test = torch.tensor(y_me_test, dtype=torch.long)


In [8]:
class MultiOutputNet(nn.Module):
    def __init__(self, input_dim, n_exercise, n_meal):
        super().__init__()

        self.shared = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU()
        )

        self.exercise_head = nn.Linear(64, n_exercise)
        self.meal_head = nn.Linear(64, n_meal)

    def forward(self, x):
        features = self.shared(x)
        return self.exercise_head(features), self.meal_head(features)

In [9]:
input_dim = X_train.shape[1]
n_exercise = len(np.unique(y_exercise))
n_meal = len(np.unique(y_meal))

model = MultiOutputNet(input_dim, n_exercise, n_meal)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)


epochs = 50

In [10]:
for epoch in range(epochs):
    model.train()

    optimizer.zero_grad()

    ex_logits, meal_logits = model(X_train)

    loss_ex = criterion(ex_logits, y_ex_train)
    loss_me = criterion(meal_logits, y_me_train)

    loss = loss_ex + 2.0 * loss_me
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")


model.eval()

Epoch 5/50, Loss: 2.8039
Epoch 10/50, Loss: 0.2792
Epoch 15/50, Loss: 0.0023
Epoch 20/50, Loss: 0.0000
Epoch 25/50, Loss: 0.0000
Epoch 30/50, Loss: 0.0000
Epoch 35/50, Loss: 0.0000
Epoch 40/50, Loss: 0.0000
Epoch 45/50, Loss: 0.0000
Epoch 50/50, Loss: 0.0000


MultiOutputNet(
  (shared): Sequential(
    (0): Linear(in_features=8, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
  )
  (exercise_head): Linear(in_features=64, out_features=4, bias=True)
  (meal_head): Linear(in_features=64, out_features=4, bias=True)
)

In [11]:
with torch.no_grad():
    ex_logits, meal_logits = model(X_test)

    ex_preds = torch.argmax(ex_logits, dim=1).numpy()
    meal_preds = torch.argmax(meal_logits, dim=1).numpy()

from sklearn.metrics import classification_report

print("Exercise Schedule:")
print(classification_report(y_ex_test, ex_preds))

print("Meal Plan:")
print(classification_report(y_me_test, meal_preds))


Exercise Schedule:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3993
           1       1.00      1.00      1.00      4219
           2       1.00      1.00      1.00      3838
           3       1.00      1.00      1.00      3950

    accuracy                           1.00     16000
   macro avg       1.00      1.00      1.00     16000
weighted avg       1.00      1.00      1.00     16000

Meal Plan:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3950
           1       1.00      1.00      1.00      4219
           2       1.00      1.00      1.00      3838
           3       1.00      1.00      1.00      3993

    accuracy                           1.00     16000
   macro avg       1.00      1.00      1.00     16000
weighted avg       1.00      1.00      1.00     16000

